# Multiple Outputs!

<font size=5><b> Y</b></font>ou see, we have done "Multiple Inputs" successfully with just appending the `Input()` extra in the list. ***But***, now for <u> Multiple Outputs </u> we can't just add the `Output()` in the list. As the syntax itself doesn't allow us to do so — but we **can**.



The syntax is not different but to give multiple outputs, we have to make a `Seperate Function` for each output.<br>
—
### Why?
\---<br>
Short Ans: **Because of the *current* syntax** <br>
Long Ans: Actully because of the current syntax which doesn't allow us to pass multiple Outputs in the list, what happens is that when we have se the `Output('id', 'property')` ← it only points to one graph or obj on the page and in addition, the function itself doesn't allow us to send seperate dicts. Only one dict can be returned (of course we can make a list of dicts but see, that is not useful on the othersize).

### How?
\---<br>
We can copy the same function again, with other graph id. Easy. This way we can handle multiple inputs, and also handle multiple outputs. Together.

**The Good News is** in the newer version (not installed in this pc) <u>has the support</u> of multiple outputs. Which can be passed in the list. Much like multiple inputs. Access it [here](https://community.plotly.com/t/multiple-outputs-in-dash-now-available/19437).

—<br>
But for now, we have to work with the old way. Let's do this.

# 

In [1]:
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output

import plotly.graph_objs as go
import plotly.offline as pyo

import pandas as pd

I am gonna use the same GDP data and will demonstrate how it works. We will have a little feel of dashboard with more inputs and so... let's go ∞

In [5]:
df = pd.read_csv("../DATA/Data/gapminderDataFiveYear.csv")
df

,country,year,pop,continent,lifeExp,gdpPercap
0,Afghanistan,1952,8425333.0,Asia,28.801,779.445314
1,Afghanistan,1957,9240934.0,Asia,30.332,820.853030
2,Afghanistan,1962,10267083.0,Asia,31.997,853.100710
3,Afghanistan,1967,11537966.0,Asia,34.020,836.197138
4,Afghanistan,1972,13079460.0,Asia,36.088,739.981106
...,...,...,...,...,...,...
1699,Zimbabwe,1987,9216418.0,Africa,62.351,706.157306
1700,Zimbabwe,1992,10704340.0,Africa,60.377,693.420786
1701,Zimbabwe,1997,11404948.0,Africa,46.809,792.449960
1702,Zimbabwe,2002,11926563.0,Africa,39.989,672.038623


### Building from scratch

In [63]:
# Simple app creation
app = dash.Dash()

# Making a list of ountries for dropdown
countries = [{"label":country, "value":country} for country in df.country.unique()]

# Making a list of metrices
metrices = [{"label":"Population", "value":"pop"},
            {"label":"LifeExp", "value":"lifeExp"},
            {"label":"GDP Per Capita", "value":"gdpPercap"}]

# A common style for dropdown DIV
style = {"display":"inline-block",
         "width":"50%"}

# Managing Layout
app.layout = html.Div([
                    html.Div([
                        html.Label("Country"),        # DRP 1
                        dcc.Dropdown(options=countries, id="countries", value="India")
                    ], style=style),
                    html.Div([
                        html.Label("Metric"),        # DRP 2
                        dcc.Dropdown(options=metrices, id="metrices", value="pop")
                    ], style=style),
                    html.Div([
                        html.Label("Year"),          # Rng Slider
                        dcc.RangeSlider(
                                id='year_slider',
                                min=df.year.min(),
                                max=df.year.max(),
                                step=1,
                                value=[df.year.min(), df.year.max()],
                                marks={year : year for year in range(df.year.min(), df.year.max()+1, 2)}
                        ),
                    ], style={"width": "80%", "margin-left":"6%", "margin-top":"2%"}),
                    html.Div([                       # Graph 1
                        dcc.Graph(id="graph-1")
                    ], style=dict(**style, **{"margin-top":"2%"})),
                    html.Div([                       # Graph 2
                        dcc.Graph(id="graph-2")
                    ], style=dict(**style, **{"margin-top":"2%"}))
])



# For bar chart (all countries)
@app.callback(Output("graph-1", "figure"),
             [Input("metrices", "value"),
              Input("year_slider", "value")])
def graph1_updater(metric, year):
    
    # Filtering according to the year range
    filtered_df = df[df.year.isin(range(year[0], year[1] + 1))]
    
    # Just grouping by all counry names so we
    # get the mean of all years in range
    filtered_df = filtered_df.groupby("country")[metric].mean()
    filtered_df = filtered_df.sort_values(ascending=False)[:20]
    
    # Then making the bars.
    traces = [go.Bar(x=filtered_df.index,
                     y=filtered_df)]
    layout = go.Layout(title=f"{metric.title()} in all countries during {year[0]} - {year[1]}",
                       xaxis=dict(title="Countries"),
                       yaxis=dict(title=f"{metric.title()}"))
    figure = go.Figure(data=traces, layout=layout)
    return figure


# For line chart (all years)
@app.callback(Output("graph-2", "figure"),
             [Input("countries", "value"),
              Input("metrices", "value")])
def graph2_updater(country, metric):

    # Filtering according to the inputs
    filtered_df = df[df.country == country]
    
    # Then making the lineplot.
    traces = [go.Scatter(x=filtered_df.year,
                         y=filtered_df[metric])]
    layout = go.Layout(title=f"{metric.title()} in {country} over the years",
                       xaxis=dict(title="← Years →"),
                       yaxis=dict(title=f"← {metric.title()} →"))
    figure = go.Figure(data=traces, layout=layout)
    return figure


app.run_server()

### Don't want to read the code?
The simple takeaways are:
- For multiple outputs we have to use different functions.
- Don't get worried about the code above, there is only a repeatation of function twice that's why it looks long.
- It is so easy.

# 

# Next up, we will...
Build another dash where we are doing something different. We will load the images according to the selections!